In [ ]:
import pandas as pd
import numpy as np
import re
import glob
import re
import nltk
from nltk.tokenize import RegexpTokenizer
import zipfile
import ast
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
liwc2 = pd.read_excel('/home_remote/dic_avg100_annotated_official.xlsx')
liwc2['Terms'] = liwc2['Term'].apply(lambda x: ast.literal_eval(x))
result = dict(zip(liwc2['Category'], liwc2['Terms']))

In [ ]:
def parse_dict(_dict):
    '''
    param: dictionary with key as categorie and value as the list of words
    return: a lexicon with key as word and values as the list of categories the word belongs to.
    '''
    lt =[]
    for x in _dict.values():
        lt = lt+x
    mylist = [*set(lt)]
    lexicon = {}
    for i in mylist:
        ls = []
        for j in _dict.keys():
            #print(j)
            if i in _dict[j]:
                ls.append(j)
        lexicon[i] = ls
    return lexicon, list(_dict.keys())
    

In [ ]:
# ref: https://github.com/chbrown/liwc-python
def build_trie(lexicon):
    """
    A `*` indicates a wildcard match.
    """
    trie = {}
    for pattern, category_names in lexicon.items():
        cursor = trie
        for char in pattern:
            if char == "*":
                cursor["*"] = category_names
                break
            if char not in cursor:
                cursor[char] = {}
            cursor = cursor[char]
        cursor["$"] = category_names
    return trie

In [ ]:
#ref: https://github.com/chbrown/liwc-python
def search_trie(trie, token, token_i=0):
    if "*" in trie:
        return trie["*"]
    if "$" in trie and token_i == len(token):
        return trie["$"]
    if token_i < len(token):
        char = token[token_i]
        if char in trie:
            return search_trie(trie[char], token, token_i + 1)
    return []

In [ ]:
# ref: https://github.com/chbrown/liwc-python
def load_token_parser(dic):
    lexicon, category_names = parse_dict(dic)
    trie = build_trie(lexicon)

    def parse_token(token):
        for category_name in search_trie(trie, token):
            yield category_name

    return parse_token, category_names

In [ ]:
def tokenize(text):
    tokenizer = RegexpTokenizer(r'\w+|\$[\d\.]+|\S+')
    for match in tokenizer.tokenize(text) :
        yield match

In [ ]:
def main(input_, result):
   # dic['Terms'] = dic['Term'].apply(lambda x: ast.literal_eval(x))
   # result = dict(zip(dic['Category'], dic['Terms']))
    #lexicon = parse_dict(result)
    parse, category_names = load_token_parser(result)
    input_tokens = tokenize(input_.lower())
    counts = Counter(category for token in input_tokens for category in parse(token))
    
#   a = pd.DataFrame.from_dict(dict(counts),orient = 'index').reset_index()
#   a= a.rename(columns={"index": "Category", 0: "Count"})
#   a['Percentage(%)']= round(a['Count']/len(input_.split(' '))*100,2)
    return counts

# LIWC alike

In [ ]:
# Load data
%store -r df
%store -r df_neg

In [ ]:
#combine Text and Title
df['Text_Title'] = df['Title'] + '.' + df['Text']
df_neg['Text_Title'] = df_neg['Title'] + '.' + df_neg['Text']
#combine all text of same user into one string
text_per_user_pos = df.groupby('User')['Text_Title'].apply(' '.join).reset_index()
text_per_user_neg = df_neg.groupby('User')['Text_Title'].apply(' '.join).reset_index()
#add label
text_per_user_pos['label'] = 1
text_per_user_neg['label'] = 0

In [ ]:
#create an empty dict
avg_pos = {}
for i in df['User'].unique():
    temp = df[df['User']==i]
    #create an empty vector with length of number of post of the user
    vec = []
    for j in temp['Text_Title']:
        vc = np.zeros(len(liwc2['Category']))
        tp = main(j, liwc2)
        tp = np.array(list(tp.values()))/len(j.split(' '))
        for k in range(len(tp)):
            vc[k] = tp[k]
        # insert vc to vec
        vec.append(vc)
    vec_avg = np.mean(vec, axis=0)
    #append the vector to the dictionary with key as user id and value as vector
    avg_pos[i] = vec_avg

In [ ]:
# input text to LIWC-alike
liwc_pos = [main(text, result) for text in text_per_user_pos['Text_Title']]
liwc_neg = [main(text, result) for text in text_per_user_neg['Text_Title']]
#then buid a dictionary with key as user, value as a list of LIWC features
liwc_pos_dict = {}
liwc_neg_dict = {}
for i in range(len(liwc_pos)):
    liwc_pos_dict[text_per_user_pos['User'][i]] = liwc_pos[i]
for i in range(len(liwc_neg)):
    liwc_neg_dict[text_per_user_neg['User'][i]] = liwc_neg[i]


In [ ]:
#calculate percentage of each LIWC feature per user
liwc_pos_percent = {}
liwc_neg_percent = {}
for key, value in liwc_pos_dict.items():
    liwc_pos_percent[key] = {k: v / sum(value.values()) for k, v in value.items()}
for key, value in liwc_neg_dict.items():
    liwc_neg_percent[key] = {k: v / sum(value.values()) for k, v in value.items()}

In [ ]:
#correlation of LIWC features with label
liwc_pos_percent_df = pd.DataFrame(liwc_pos_percent).T
liwc_neg_percent_df = pd.DataFrame(liwc_neg_percent).T
liwc_pos_percent_df['label'] = 1
liwc_neg_percent_df['label'] = 0
liwc_percent_df = pd.concat([liwc_pos_percent_df, liwc_neg_percent_df])
#liwc_percent_df = liwc_percent_df.fillna(0)
corr = liwc_percent_df.corr()

In [ ]:
#choose the top 15 most correlated features to label
corr_label = corr['label'].sort_values(ascending=False)
corr_label = corr_label[1:16]
corr_label

In [ ]:
#get the name of top 15 features
liwc_features = list(corr_label.index)
#concat with label
liwc_features.append('label')

In [ ]:
plt.figure(figsize=(10,10))

corr_liwc = liwc_percent_df[liwc_features].corr()
sns.heatmap(corr_liwc, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.show()